In [47]:
import pandas as pd
import numpy as np

In [48]:
# File paths
file1 = r"D:\Projects\Hotel-Analytics-RAG\dataset\hotel_bookings_1.csv"
file2 = r"D:\Projects\Hotel-Analytics-RAG\dataset\hotel_bookings_2.csv"

In [49]:
# Load the datasets
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

In [50]:
print(f"File 1 shape: {df1.shape}")
print(f"File 2 shape: {df2.shape}")

File 1 shape: (119390, 32)
File 2 shape: (119390, 32)


In [51]:
def clean_dataset(df, name=""):
    original_shape = df.shape
    
    # Strip whitespace from column names
    df.columns = df.columns.str.strip()
    
    # Strip strings and standardize text columns
    for col in df.select_dtypes(include='object'):
        df[col] = df[col].str.strip()
    
    # Fill commonly missing fields
    fill_values = {
        'children': 0,
        'country': 'unknown',
        'agent': 'unknown',
        'company': 'unknown'
    }
    df.fillna(value=fill_values, inplace=True)

    # Fill remaining numeric nulls with median
    for col in df.select_dtypes(include=[np.number]):
        if df[col].isnull().sum() > 0:
            df[col].fillna(df[col].median(), inplace=True)
    
    # Drop rows with all values missing (if any)
    df.dropna(how='all', inplace=True)
    
    missing_after = df.isnull().sum().sum()

    print(f"\n{name} - Cleaning Summary:")
    print(f"Original shape: {original_shape}")
    print(f"Remaining missing values: {missing_after}")
    
    return df

In [52]:
df1_clean = clean_dataset(df1, name="File 1")
df2_clean = clean_dataset(df2, name="File 2")


File 1 - Cleaning Summary:
Original shape: (119390, 32)
Remaining missing values: 0

File 2 - Cleaning Summary:
Original shape: (119390, 32)
Remaining missing values: 0


In [53]:
combined_df = pd.concat([df1_clean, df2_clean], ignore_index=True)
print(f"\nCombined shape before dropping duplicates: {combined_df.shape}")


Combined shape before dropping duplicates: (238780, 32)


In [54]:
combined_df.drop_duplicates(inplace=True)
print(f"Shape after dropping duplicates: {combined_df.shape}")

Shape after dropping duplicates: (174792, 32)


In [55]:
int_cols = ['adults', 'children', 'babies']
for col in int_cols:
    if col in combined_df.columns:
        combined_df[col] = combined_df[col].astype(int)

In [56]:
combined_df['reservation_status_date'] = pd.to_datetime(
    combined_df['reservation_status_date'], errors='coerce'
)

In [57]:
combined_df['total_nights'] = (
    combined_df['stays_in_weekend_nights'] + combined_df['stays_in_week_nights']
)
combined_df = combined_df[combined_df['total_nights'] > 0]
print(f"Shape after removing 0-night stays: {combined_df.shape}")

Shape after removing 0-night stays: (173490, 33)


In [58]:
combined_df = combined_df[combined_df['adr'] < 500]
print(f"Shape after removing outlier ADR values: {combined_df.shape}")

Shape after removing outlier ADR values: (173484, 33)


In [59]:
null_summary = combined_df.isnull().sum()
null_columns = null_summary[null_summary > 0]
print("\nRemaining nulls per column (if any):")
print(null_columns)


Remaining nulls per column (if any):
reservation_status_date    86742
dtype: int64


In [60]:
month_map = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4,
    'May': 5, 'June': 6, 'July': 7, 'August': 8,
    'September': 9, 'October': 10, 'November': 11, 'December': 12
}
combined_df['arrival_month_num'] = combined_df['arrival_date_month'].map(month_map)

In [61]:
combined_df['arrival_date'] = pd.to_datetime(
    dict(
        year=combined_df['arrival_date_year'],
        month=combined_df['arrival_month_num'],
        day=combined_df['arrival_date_day_of_month']
    ),
    errors='coerce'
)


In [62]:
combined_df['revenue'] = combined_df['adr'] * combined_df['total_nights']

In [63]:
print("\nFinal Columns Overview:")
print(combined_df.columns.tolist())

print("\nNulls remaining (if any):")
print(combined_df.isnull().sum().sort_values(ascending=False).head(10))


Final Columns Overview:
['hotel', 'is_canceled', 'lead_time', 'arrival_date_year', 'arrival_date_month', 'arrival_date_week_number', 'arrival_date_day_of_month', 'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children', 'babies', 'meal', 'country', 'market_segment', 'distribution_channel', 'is_repeated_guest', 'previous_cancellations', 'previous_bookings_not_canceled', 'reserved_room_type', 'assigned_room_type', 'booking_changes', 'deposit_type', 'agent', 'company', 'days_in_waiting_list', 'customer_type', 'adr', 'required_car_parking_spaces', 'total_of_special_requests', 'reservation_status', 'reservation_status_date', 'total_nights', 'arrival_month_num', 'arrival_date', 'revenue']

Nulls remaining (if any):
reservation_status_date      86742
hotel                            0
lead_time                        0
is_canceled                      0
arrival_date_year                0
arrival_date_month               0
arrival_date_day_of_month        0
arrival_date_week_nu

In [64]:
output_path = "hotel_bookings_dataset.csv"
combined_df.to_csv(output_path, index=False)
print(f"\n Preprocessed dataset ready for analytics saved to: {output_path}")
print(f"Final dataset shape: {combined_df.shape}")



 Preprocessed dataset ready for analytics saved to: hotel_bookings_dataset.csv
Final dataset shape: (173484, 36)


In [65]:
print("\n--- Sample of Cleaned Dataset ---")
display(combined_df.head())


--- Sample of Cleaned Dataset ---


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,total_nights,arrival_month_num,arrival_date,revenue
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,Transient,75.0,0,0,Check-Out,2015-07-02,1,7,2015-07-01,75.0
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,Transient,75.0,0,0,Check-Out,2015-07-02,1,7,2015-07-01,75.0
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,Transient,98.0,0,1,Check-Out,2015-07-03,2,7,2015-07-01,196.0
6,Resort Hotel,0,0,2015,July,27,1,0,2,2,...,Transient,107.0,0,0,Check-Out,2015-07-03,2,7,2015-07-01,214.0
7,Resort Hotel,0,9,2015,July,27,1,0,2,2,...,Transient,103.0,0,1,Check-Out,2015-07-03,2,7,2015-07-01,206.0
